## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

### PromptTemplate


In [2]:
from langchain.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [3]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [4]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [5]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

### Model(LLM)

`ChatOpenAI`

OpenAI 사의 채팅 전용 Large Language Model(llm) 입니다.

객체를 생성할 때 다음을 옵션 값을 지정할 수 있습니다. 옵션에 대한 상세 설명은 다음과 같습니다.

`temperature`

- 사용할 샘플링 온도는 0과 2 사이에서 선택합니다. 0.8과 같은 높은 값은 출력을 더 무작위하게 만들고, 0.2와 같은 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.

`max_tokens`

- 채팅 완성에서 생성할 토큰의 최대 개수입니다.

`model_name`: 적용 가능한 모델 리스트

- 링크: https://www.teddynote.com/openai/openai-models/
- 링크: https://platform.openai.com/docs/models


In [6]:
# from langchain_openai import ChatOpenAI

# model = ChatOpenAI(
#     model="gpt-3.5-turbo",
#     max_tokens=2048,
#     temperature=0.1,
# )
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-pro")

/opt/miniconda3/envs/ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Output


In [7]:
from langchain_core.output_parsers import StrOutputParser

output = StrOutputParser()

## Chain 생성

**LCEL(LangChain Expression Language)**

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달되며, 그 다음 모델 출력은 출력 파서로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [9]:
# 주어진 나라에 대하여 수도를 묻는 프롬프트 템플릿을 생성합니다.
# prompt = PromptTemplate.from_template("{country}의 수도는 어디인가요?")

# 주어진 나라에 대하여 수도를 묻는 프롬프트 템플릿을 생성합니다.
template = """
당신은 친절하게 답변해 주는 친절 봇입니다. 사용자의 질문에 [FORMAT]에 맞추어 답변해 주세요.
답변은 항상 한글로 작성해 주세요.

질문:
{question}에 대하여 설명해 주세요.

FORMAT:
- 개요:
- 예시:
- 출처:
"""

template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 [FORMAT]에 영어 회화를 작성해 주세요.

상황:
{question}

FORMAT:
- 영어 회화:
- 한글 해석:
"""


prompt = PromptTemplate.from_template(template)


# OpenAI 챗모델을 초기화합니다.
# model = ChatOpenAI(model="gpt-4-turbo-preview")
# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

# 완성된 Chain 을 이용하여 country 를 '대한민국'으로 설정하여 실행합니다.
# chain.invoke({"country": "대한민국"})
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

**영어 회화:**

* "Excuse me, can I see a menu, please?"
* "I'd like to order the grilled salmon with roasted vegetables."
* "Can I get a side of mashed potatoes with that?"
* "And a glass of water, please."

**한글 해석:**

* "죄송합니다. 메뉴를 보여주실 수 있나요?"
* "저는 구운 연어와 구운 채소를 주문하고 싶습니다."
* "그런데 으깬 감자도 곁들여 주실 수 있나요?"
* "물 한 잔 부탁드립니다."


In [10]:
# 완성된 Chain 을 이용하여 question 를 '미국에서 피자 주문'으로 설정하여 실행합니다.
print(chain.invoke({"question": "미국에서 피자 주문"}))

**영어 회화:**

**피자 주문자:** Hello, I'd like to order a pizza, please.

**피자 가게 직원:** Sure, what kind of pizza would you like?

**피자 주문자:** I'll have a large pepperoni pizza with extra cheese.

**피자 가게 직원:** Okay, and what toppings would you like on that?

**피자 주문자:** Just pepperoni and extra cheese.

**피자 가게 직원:** Got it. And what size would you like?

**피자 주문자:** Large.

**피자 가게 직원:** And is that for delivery or pickup?

**피자 주문자:** Delivery, please.

**피자 가게 직원:** Okay, and what's your address?

**피자 주문자:** It's 123 Main Street.

**피자 가게 직원:** Perfect. And what's your phone number?

**피자 주문자:** It's 555-123-4567.

**피자 가게 직원:** Thank you. Your pizza will be there in about 30 minutes.

**피자 주문자:** Great. Thanks!

**한글 해석:**

**피자 주문자:** 안녕하세요. 피자 주문하고 싶은데요.

**피자 가게 직원:** 물론이죠. 어떤 피자를 주문하시겠어요?

**피자 주문자:** 페퍼로니 피자 대형으로 주문하고 싶은데, 치즈를 좀 더 많이 넣어주세요.

**피자 가게 직원:** 알겠습니다. 그 외에 토핑은 뭔가 얹어드릴까요?

**피자 주문자:** 페퍼로니와 치즈만 넣어주세요.

**피자 가게 직원:** 알았습니다. 크기는요?

**피자 주문자:** 대형으로요.

**피자 가게 직원:** 배달이나 직접 찾

## 전체 파이프라인

단계를 따라가려면:

1. 원하는 주제에 대한 사용자 입력을 `{"topic": "양자역학"}`로 전달합니다.
2. `prompt` 컴포넌트는 사용자 입력을 받아 `topic`을 사용하여 프롬프트를 구성한 후 PromptValue를 생성합니다.
3. `model` 컴포넌트는 생성된 프롬프트를 가져와 OpenAI LLM 모델에 평가를 위해 전달합니다. 모델에서 생성된 출력은 `ChatMessage` 객체입니다.
4. 마지막으로, `output_parser` 컴포넌트는 `ChatMessage`를 받아 이를 Python 문자열로 변환하며, 이는 invoke 메서드에서 반환됩니다.

`prompt` 또는 `prompt | model`과 같은 체인의 작은 버전을 테스트하여 중간 결과를 확인하고자 하는 경우에 대해 유의하세요.

`prompt` 객체와 `model` 객체를 사용하여 대화형 AI 시스템을 시뮬레이션합니다. 먼저, `prompt.invoke` 메서드를 호출하여 사용자 입력에 대한 프롬프트 값을 생성합니다. 그런 다음, `prompt` 객체와 `model` 객체를 파이프 연산자(`|`)로 연결하고 `invoke` 메서드를 호출하여 AI 모델이 생성한 응답을 반환합니다.

이 과정에서 AI 모델은 '양자역학'이라는 주제에 대한 쉽게 이해할 수 있는 답변을 생성합니다.


![](./images/lcel-flow.png)


In [11]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

# input 딕셔너리에 주제를 'ice cream'으로 설정합니다.
input = {"topic": "양자역학"}

# prompt 객체의 invoke 메서드를 사용하여 input을 전달하고 대화형 프롬프트 값을 생성합니다.
prompt.invoke(input)

# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
(prompt | model).invoke(input)

AIMessage(content='**양자역학이란?**\n\n양자역학은 원자와 아원자 입자의 행동을 설명하는 물리학의 한 분야입니다.\n\n**핵심 개념:**\n\n* **파동-입자 이중성:** 입자는 파동과 같은 성질을 가지기도 하고 입자와 같은 성질을 가지기도 합니다.\n* **불확정성 원리:** 입자의 위치와 운동량을 동시에 정확하게 알 수 없습니다.\n* **양자화:** 에너지, 각운동량과 같은 물리량은 불연속적인 "양자"로만 존재할 수 있습니다.\n\n**양자역학의 영향:**\n\n양자역학은 우리가 우주를 이해하는 방식에 혁명을 일으켰습니다.\n\n* **원자 구조:** 양자역학은 원자의 전자 분포를 설명하여 화학과 물리학의 토대를 마련했습니다.\n* **고체물리:** 양자역학은 전자의 성질을 이해하는 데 도움이 되었고, 이를 통해 반도체, 초전도체 등의 새로운 재료 개발로 이어졌습니다.\n* **핵물리:** 양자역학은 핵 반응을 이해하는 데 필수적이며, 핵 에너지와 핵무기 개발로 이어졌습니다.\n* **양자 컴퓨팅:** 양자역학을 활용하여 전통적인 컴퓨터보다 훨씬 빠르고 강력한 컴퓨터를 개발하려고 노력하고 있습니다.\n\n**양자역학의 이해하기 어려운 측면:**\n\n양자역학은 고전 물리학과 크게 다르며 이해하기 어려울 수 있습니다.\n\n* **확률적 성질:** 양자역학에서는 결과가 확률적입니다. 즉, 특정 결과의 발생 가능성은 예측할 수 있지만 확실하게 알 수는 없습니다.\n* **파동 함수:** 파동 함수는 입자의 상태를 나타내는 수학적 함수입니다. 파동 함수는 입자의 위치나 운동량과 같은 물리량의 확률 분포를 제공합니다.\n* **얽힘:** 두 개 이상의 입자가 서로 얽히면 하나의 입자에 대한 변화가 다른 입자에도 즉시 영향을 미칩니다. 이러한 현상은 직관에 어긋납니다.\n\n전반적으로 양자역학은 우리가 우주를 이해하는 데 핵심적인 분야이며, 기술 발전과 과학적 이해에 엄청난 영향을 미쳤습니다.', response_metadata={'prom

In [13]:
# parse_output 메서드를 사용하여 AI 모델이 생성한 메시지 문자열로 출력합니다.
(prompt | model | output_parser).invoke(input)

"양자역학은 물리학의 한 분야로, 아주 작은 입자들의 세계에서 일어나는 현상을 설명하는 이론입니다. 일상생활에서는 경험하기 어려운, 원자나 전자와 같은 미소한 입자들의 행동을 다룹니다. 양자역학을 이해하기 위해서는 몇 가지 중요한 개념을 알아야 합니다.\n\n1. **양자**: '양자'란 '양'을 의미하는데, 여기서는 에너지가 불연속적인 최소 단위로 존재한다는 개념입니다. 즉, 에너지가 연속적인 흐름이 아니라 '양자'라고 불리는 일정한 '덩어리'로만 전달되거나 흡수된다는 뜻입니다.\n\n2. **파동-입자 이중성**: 양자역학에서는 빛과 같은 입자들이 때로는 파동처럼, 때로는 입자처럼 행동한다는 것을 설명합니다. 즉, 같은 것이 상황에 따라 파동이 될 수도 있고 입자가 될 수도 있다는 것입니다.\n\n3. **불확정성 원리**: 하이젠베르크의 불확정성 원리에 따르면, 우리는 입자의 위치와 운동량(속도와 방향)을 동시에 정확하게 알 수 없습니다. 한 쪽을 더 정확하게 알려고 하면 다른 한 쪽은 더 불확실해집니다. 이는 양자 세계에서의 불확실성이 근본적인 성질임을 의미합니다.\n\n4. **양자 중첩**: 입자가 두 가지 이상의 상태에 동시에 존재할 수 있다는 개념입니다. 예를 들어, 전자가 동시에 여러 위치에 존재할 수 있습니다. 이 상태는 관측될 때까지 유지되며, 관측하는 순간 하나의 상태로 '붕괴'됩니다.\n\n5. **양자 얽힘**: 두 개 이상의 입자가 서로 얽혀서 한 입자의 상태를 결정하면 즉시 다른 입자의 상태도 결정되는 현상입니다. 이는 입자들이 서로 멀리 떨어져 있어도 적용되며, 양자역학의 가장 신비로운 부분 중 하나입니다.\n\n양자역학은 이와 같은 비직관적이고 독특한 원리들로 인해 초기에는 많은 논란을 일으켰습니다. 하지만 현재는 물리학의 근간을 이루는 중요한 이론으로, 컴퓨터, 통신, 의료 등 다양한 분야에서 응용되고 있습니다."